In [1]:
from scipy.io import loadmat
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
def load_subject_data(index):

    ind_str = ""
    if index < 10: ind_str = "00"+str(index)
    if index >= 10: ind_str = "0"+str(index)

    path = "..\dataset\derivatives\sub-" + ind_str + "\eeg\sub-" + ind_str + "_task-eyesclosed_eeg.set"
    data = loadmat(path)

    return data

In [3]:
%matplotlib qt

In [4]:
def plot_sub_data(data, window):
    one_chanel = data['data'][1]
    times = data['times'].T

    plt.plot(times, one_chanel)
    plt.show()

window = [5000, 10000]
data = load_subject_data(17)
plot_sub_data(data, window)
data = load_subject_data(37)
plot_sub_data(data, window)

In [5]:
mmse = pd.read_csv('..\dataset\participants.tsv', sep='\t')

subject_list = mmse['participant_id']
means = np.zeros((len(subject_list), 1))
stds = np.zeros((len(subject_list), 1))

mmse_score = mmse['MMSE']

for sub in subject_list:
    sub_int = int(sub[4:])
    data = load_subject_data(sub_int)
    eeg = data['data'][0]

    means[sub_int-1] = np.mean(eeg)
    stds[sub_int-1] = np.std(eeg)

plt.scatter(mmse_score, means)
plt.show()
plt.scatter(mmse_score, stds)
plt.show()

In [6]:
usefull_keys = ['data', 'times', 'group']

def create_dict_subject(index, usefull_keys):
    usefull_data_dict = {}
    data = load_subject_data(index)

    for key in usefull_keys:
        if key == 'data':
            for i in range(data[key].shape[0]):
                usefull_data_dict['ch' + str(i)] = [data[key][i].T]
        elif key == 'times':
            usefull_data_dict[key] = [np.squeeze(data[key])]
        elif key == 'group':
            usefull_data_dict[key] = data[key][0]
        else:
            usefull_data_dict[key] = [data[key]]

    mmse = pd.read_csv('..\dataset\participants.tsv', sep='\t')
    mmse_score = mmse['MMSE']

    usefull_data_dict['MMSE'] = mmse_score[index-1]

    return usefull_data_dict

In [7]:
n_subjects = len(subject_list)

def save_data(path):
    df = pd.DataFrame(columns = ['ch'+str(i) for i in range(19)] + ['times','group','MMSE'])
    for i in range(n_subjects):
        subject_data_dict = create_dict_subject(i+1, usefull_keys)
        df = df.append(subject_data_dict, ignore_index=True)
    print(df.head())
    df.to_csv(path)
    
# save_data("..\\all_subjects.csv")

In [ ]:
def create_data_matrix(index):
    data = load_subject_data(index)